In [163]:
import numpy as np

import pandas as pd

from nltk.tokenize import word_tokenize

import re

import nltk

from contractions import contractions_dict

from nltk.corpus import stopwords

from spacy.lang.en.stop_words import STOP_WORDS

from itertools import filterfalse

from nltk import pos_tag

from nltk.stem import WordNetLemmatizer

from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.decomposition import PCA

from sklearn.datasets import load_files

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics._classification import classification_report

#nltk.download('averaged_perceptron_tagger')

#nltk.download('punkt')
#nltk.download('stopwords')

In [164]:
data_load = load_files('mini_newsgroups',encoding='latin1',decode_error='ignore')

In [165]:
data = pd.DataFrame({
    'text': data_load.data,
    'target': data_load.target,
    'category': [data_load.target_names[i] for i in data_load.target]
})

In [166]:
data

,text,target,category
0,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,4,comp.sys.mac.hardware
1,Newsgroups: sci.crypt\nPath: cantaloupe.srv.cs...,11,sci.crypt
2,"Newsgroups: sci.crypt,alt.privacy.clipper\nPat...",11,sci.crypt
3,Newsgroups: rec.autos\nPath: cantaloupe.srv.cs...,7,rec.autos
4,Xref: cantaloupe.srv.cs.cmu.edu soc.culture.ar...,17,talk.politics.mideast
...,...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu rec.motorcycle...,8,rec.motorcycles
1996,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,12,sci.electronics
1997,Newsgroups: talk.politics.guns\nPath: cantalou...,16,talk.politics.guns
1998,Path: cantaloupe.srv.cs.cmu.edu!rochester!udel...,5,comp.windows.x


In [167]:
data_load.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [168]:
data['category'].unique()

array(['comp.sys.mac.hardware', 'sci.crypt', 'rec.autos',
       'talk.politics.mideast', 'soc.religion.christian', 'sci.med',
       'talk.politics.misc', 'alt.atheism', 'sci.electronics',
       'comp.os.ms-windows.misc', 'comp.graphics', 'talk.religion.misc',
       'rec.sport.hockey', 'comp.windows.x', 'rec.sport.baseball',
       'talk.politics.guns', 'comp.sys.ibm.pc.hardware',
       'rec.motorcycles', 'sci.space', 'misc.forsale'], dtype=object)

In [169]:
data['text'][0]

"Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!usc!cs.utexas.edu!uunet!pipex!marble.uknet.ac.uk!uknet!acorn!camcon!aw\nFrom: aw@camcon.co.uk (Alain Waha)\nNewsgroups: comp.sys.mac.hardware\nSubject: Re: New Duo Dock info.\nMessage-ID: <30003@io.camcon.co.uk>\nDate: 6 Apr 93 08:55:26 GMT\nReferences: <nazario-040493023201@stiles-42-kstar-node.net.yale.edu> <1993Apr4.164142.14744@umiami.ir.miami.edu> <nazario-050493004102@stiles-42-kstar-node.net.yale.edu>\nOrganization: Cambridge Consultants Ltd., Cambridge, UK\nLines: 7\n\n>> In article <nazario-040493023201@stiles-42-kstar-node.net.yale.edu>, nazario@pop.cis.yale.edu (Edgardo Nazario) writes:\n>> > The info I am about to give is not a rumour, it's the truth. The new\n>> > macintosh coming in the second quarter, will have a cpu of their own. \n\nExcuse me but... have not all Macs got a CPU!!!\n\nAlain\n"

In [170]:
data['category'].value_counts()

category
comp.sys.mac.hardware       100
sci.crypt                   100
sci.space                   100
rec.motorcycles             100
comp.sys.ibm.pc.hardware    100
talk.politics.guns          100
rec.sport.baseball          100
comp.windows.x              100
rec.sport.hockey            100
talk.religion.misc          100
comp.graphics               100
comp.os.ms-windows.misc     100
sci.electronics             100
alt.atheism                 100
talk.politics.misc          100
sci.med                     100
soc.religion.christian      100
talk.politics.mideast       100
rec.autos                   100
misc.forsale                100
Name: count, dtype: int64

In [171]:
data.isna().sum()

text        0
target      0
category    0
dtype: int64

In [172]:
data.tail(50)

,text,target,category
1950,Newsgroups: sci.electronics\nPath: cantaloupe....,12,sci.electronics
1951,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,13,sci.med
1952,Newsgroups: comp.sys.mac.hardware\nPath: canta...,4,comp.sys.mac.hardware
1953,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,9,rec.sport.baseball
1954,Xref: cantaloupe.srv.cs.cmu.edu comp.org.eff.t...,11,sci.crypt
1955,Newsgroups: misc.forsale\nPath: cantaloupe.srv...,6,misc.forsale
1956,Newsgroups: rec.autos\nPath: cantaloupe.srv.cs...,7,rec.autos
1957,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,19,talk.religion.misc
1958,Xref: cantaloupe.srv.cs.cmu.edu rec.audio.car:...,12,sci.electronics
1959,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54...,0,alt.atheism


In [173]:
data.duplicated().sum()

0

In [174]:
data['text'].iloc[5]

"Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!agate!doc.ic.ac.uk!warwick!qmw-dcs!qmw!demon!gtoal.com!gtoal\nNewsgroups: sci.crypt\nFrom: gtoal@gtoal.com (Graham Toal)\nSubject: Re: Off the shelf cheap DES keyseach machine (Was: Re: Corporate acceptance of the wiretap chip)\nDate: Wed, 21 Apr 1993 10:24:28 +0000\nMessage-ID: <9304211024.AA21842@pizzabox.demon.co.uk>\nSender: usenet@demon.co.uk\nLines: 16\n\n: At the company I worked for previously, I received a file that was des encryped\n: and the person that had sent it, went on vaction.  Rather than wait two weeks\n: I set up a straight frontal attack with one key at a time.  It only took two(2)\n: days to crack the file.  No, I don't have any faith in DES.\n\n: A.G.\n\nFun!  Three questions:\n\n1) What hardware did you use?\n2) When you found the key, was there anything about it that was special\n   that meant you had been lucky to find it early? (Like the first 30 bits\n   all being 0)\n3) ... or did

In [175]:
def title_striping(text):
    lines = text.split('\n')
    for i, line in enumerate(lines):
        if line.strip() == '':
            return '\n'.join(lines[i+1:]).strip()
    return text.strip() 

In [176]:
data['text'] = data['text'].apply(lambda x: title_striping(x))

In [177]:
data.head()

,text,target,category
0,>> In article <nazario-040493023201@stiles-42-...,4,comp.sys.mac.hardware
1,Deeply grateful for citations to any papers on...,11,sci.crypt
2,"The system, or 'family', key would appear to b...",11,sci.crypt
3,In <C5Dq2n.5uB@news.cso.uiuc.edu> mwbg9715@uxa...,7,rec.autos
4,In article <2BDAD779.24910@news.service.uci.ed...,17,talk.politics.mideast


In [178]:
data['text'] = data['text'].apply(lambda x: word_tokenize(x))

In [179]:
data

,text,target,category
0,"[>, >, In, article, <, nazario-040493023201, @...",4,comp.sys.mac.hardware
1,"[Deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[The, system, ,, or, 'family, ', ,, key, would...",11,sci.crypt
3,"[In, <, C5Dq2n.5uB, @, news.cso.uiuc.edu, >, m...",7,rec.autos
4,"[In, article, <, 2BDAD779.24910, @, news.servi...",17,talk.politics.mideast
...,...,...,...
1995,"[Anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[Does, anyone, know, what, causes, the, ever-g...",12,sci.electronics
1997,"[In, article, <, 1993Apr23.120935.21848, @, ic...",16,talk.politics.guns
1998,"[Is, there, any, FAQ, list, for, Programming, ...",5,comp.windows.x


In [180]:
def token_normalizer(tokens_list):
    return map(lambda x: x.lower(),tokens_list)

In [181]:
data['text'] = data['text'].apply(lambda x: token_normalizer(x))

In [182]:
data['text'] = data['text'].apply(lambda x: list(x))

In [183]:
data

,text,target,category
0,"[>, >, in, article, <, nazario-040493023201, @...",4,comp.sys.mac.hardware
1,"[deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[the, system, ,, or, 'family, ', ,, key, would...",11,sci.crypt
3,"[in, <, c5dq2n.5ub, @, news.cso.uiuc.edu, >, m...",7,rec.autos
4,"[in, article, <, 2bdad779.24910, @, news.servi...",17,talk.politics.mideast
...,...,...,...
1995,"[anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[does, anyone, know, what, causes, the, ever-g...",12,sci.electronics
1997,"[in, article, <, 1993apr23.120935.21848, @, ic...",16,talk.politics.guns
1998,"[is, there, any, faq, list, for, programming, ...",5,comp.windows.x


In [184]:
def word_expansion_contracted(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [185]:
def contraction_expansion(tokens_list):
    return map(word_expansion_contracted,tokens_list)

In [186]:
data['text'] = data['text'].apply(lambda x: contraction_expansion(x))

In [187]:
data['text'] = data['text'].apply(lambda x: list(x))

In [188]:
data

,text,target,category
0,"[>, >, in, article, <, nazario-040493023201, @...",4,comp.sys.mac.hardware
1,"[deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[the, system, ,, or, 'family, ', ,, key, would...",11,sci.crypt
3,"[in, <, c5dq2n.5ub, @, news.cso.uiuc.edu, >, m...",7,rec.autos
4,"[in, article, <, 2bdad779.24910, @, news.servi...",17,talk.politics.mideast
...,...,...,...
1995,"[anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[does, anyone, know, what, causes, the, ever-g...",12,sci.electronics
1997,"[in, article, <, 1993apr23.120935.21848, @, ic...",16,talk.politics.guns
1998,"[is, there, any, faq, list, for, programming, ...",5,comp.windows.x


In [189]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [190]:
def check_waste_words(token):
    return re.search(regex,token)

In [191]:
def filter_waste_words(tokens_list):
    return filterfalse(check_waste_words,tokens_list)

In [192]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x))

In [193]:
data['text'] = data['text'].apply(lambda x: list(x))

In [194]:
data

,text,target,category
0,"[in, article, nazario, edgardo, nazario, write...",4,comp.sys.mac.hardware
1,"[deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[the, system, or, key, would, appear, to, be, ...",11,sci.crypt
3,"[in, mark, wayne, blunier, writes, jimf, jim, ...",7,rec.autos
4,"[in, article, tclock, tim, clock, writes, in, ...",17,talk.politics.mideast
...,...,...,...
1995,"[anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[does, anyone, know, what, causes, the, black,...",12,sci.electronics
1997,"[in, article, kdw, kenneth, whitehead, writes,...",16,talk.politics.guns
1998,"[is, there, any, faq, list, for, programming, ...",5,comp.windows.x


In [195]:
def split(tokens_list):
    return map(lambda x: re.split(regex,x)[0],tokens_list)

In [196]:
data['text'] = data['text'].apply(lambda x: split(x))

In [197]:
data['text'] = data['text'].apply(lambda x: list(x))

In [198]:
data['text'][0]

['in',
 'article',
 'nazario',
 'edgardo',
 'nazario',
 'writes',
 'the',
 'info',
 'i',
 'am',
 'about',
 'to',
 'give',
 'is',
 'not',
 'a',
 'rumour',
 'it',
 'the',
 'truth',
 'the',
 'new',
 'macintosh',
 'coming',
 'in',
 'the',
 'second',
 'quarter',
 'will',
 'have',
 'a',
 'cpu',
 'of',
 'their',
 'own',
 'excuse',
 'me',
 'but',
 'have',
 'not',
 'all',
 'macs',
 'got',
 'a',
 'cpu',
 'alain']

In [199]:
eng_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [200]:
len(eng_stop_words)

401

In [201]:
def is_valid_token(token):
    return not(token in eng_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [202]:
def stopword_removal(tokens_list):
    return filter(is_valid_token,tokens_list)

In [203]:
data['text'] = data['text'].apply(lambda x: stopword_removal(x))

In [204]:
data['text'] = data['text'].apply(lambda x: list(x))

In [205]:
data

,text,target,category
0,"[article, nazario, edgardo, nazario, writes, i...",4,comp.sys.mac.hardware
1,"[deeply, grateful, citations, papers, electron...",11,sci.crypt
2,"[system, key, appear, cryptographically, usele...",11,sci.crypt
3,"[mark, wayne, blunier, writes, jimf, jim, fros...",7,rec.autos
4,"[article, tclock, tim, clock, writes, article,...",17,talk.politics.mideast
...,...,...,...
1995,"[interesting, mailing, list, bikes, lifestyle,...",8,rec.motorcycles
1996,"[know, causes, black, border, edges, computer,...",12,sci.electronics
1997,"[article, kdw, kenneth, whitehead, writes, day...",16,talk.politics.guns
1998,"[faq, list, programming, windows, thankx, info...",5,comp.windows.x


In [206]:
data['text'][0]

['article',
 'nazario',
 'edgardo',
 'nazario',
 'writes',
 'info',
 'rumour',
 'truth',
 'new',
 'macintosh',
 'coming',
 'second',
 'quarter',
 'cpu',
 'excuse',
 'macs',
 'got',
 'cpu',
 'alain']

In [207]:
tag_mapping = {'J': wordnet.ADJ, 'V': wordnet.VERB, 'N': wordnet.NOUN, 'R': wordnet.ADV}

In [208]:
def get_pos_tag_with_wordnet(tokens):
    return [(word, tag_mapping.get(tag[0], wordnet.NOUN)) for word, tag in pos_tag(tokens)]

In [209]:
data['text'] = data['text'].apply(lambda x: get_pos_tag_with_wordnet(x))

In [210]:
data

,text,target,category
0,"[(article, n), (nazario, n), (edgardo, n), (na...",4,comp.sys.mac.hardware
1,"[(deeply, r), (grateful, a), (citations, n), (...",11,sci.crypt
2,"[(system, n), (key, a), (appear, v), (cryptogr...",11,sci.crypt
3,"[(mark, n), (wayne, n), (blunier, n), (writes,...",7,rec.autos
4,"[(article, n), (tclock, n), (tim, n), (clock, ...",17,talk.politics.mideast
...,...,...,...
1995,"[(interesting, v), (mailing, v), (list, n), (b...",8,rec.motorcycles
1996,"[(know, a), (causes, v), (black, a), (border, ...",12,sci.electronics
1997,"[(article, n), (kdw, n), (kenneth, v), (whiteh...",16,talk.politics.guns
1998,"[(faq, a), (list, n), (programming, v), (windo...",5,comp.windows.x


In [211]:
lemmatizer = WordNetLemmatizer()

In [212]:
def token_lemmatization(token):
    if token == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token[0],pos=token[1])

In [213]:
def lemmatization(tokens_list):
    if len(tokens_list) > 0:
        return map(lambda x: token_lemmatization(x), tokens_list)
    else:
        return []

In [214]:
data['text'] = data['text'].apply(lambda x: lemmatization(x))

In [215]:
data['text'] = data['text'].apply(lambda x: list(x))

In [216]:
data

,text,target,category
0,"[article, nazario, edgardo, nazario, write, in...",4,comp.sys.mac.hardware
1,"[deeply, grateful, citation, paper, electronic...",11,sci.crypt
2,"[system, key, appear, cryptographically, usele...",11,sci.crypt
3,"[mark, wayne, blunier, write, jimf, jim, frost...",7,rec.autos
4,"[article, tclock, tim, clock, write, article, ...",17,talk.politics.mideast
...,...,...,...
1995,"[interest, mail, list, bike, lifestyle, politi...",8,rec.motorcycles
1996,"[know, cause, black, border, edge, computer, s...",12,sci.electronics
1997,"[article, kdw, kenneth, whitehead, writes, day...",16,talk.politics.guns
1998,"[faq, list, program, window, thankx, info, bye...",5,comp.windows.x


In [217]:
data['text'][1]

['deeply',
 'grateful',
 'citation',
 'paper',
 'electronic',
 'cash',
 'scheme',
 'enquire',
 'mind',
 'eric',
 'weaver',
 'sony',
 'avtc',
 'river',
 'oaks',
 'pkwy',
 'm',
 'sj',
 'chief',
 'engineer',
 'kfjc',
 'foothill',
 'college',
 'los',
 'alto',
 'hill']

In [218]:
vocab = set()
for tokens in data['text']:
    if tokens:
        vocab.update(tokens)

In [219]:
vocab = list(vocab)

In [220]:
vocab

['mick',
 'prominent',
 'gaza',
 'wellington',
 'synchro',
 'wb',
 'samuel',
 'dire',
 'chmod',
 'jmp',
 'peg',
 'armored',
 'lamps',
 'worshipper',
 'atal',
 'lunacy',
 'reto',
 'postet',
 'teneis',
 'sbradley',
 'invasion',
 'hobble',
 'gallagher',
 'creationists',
 'gutting',
 'hamishmar',
 'workmanship',
 'accurately',
 'polex',
 'seal',
 'gqubv',
 'classy',
 'optpublisher',
 'bxl',
 'soso',
 'experimental',
 'inadvertently',
 'harleys',
 'lookit',
 'cnes',
 'decvax',
 'asha',
 'notch',
 'teneo',
 'anni',
 'seeker',
 'ballentine',
 'hockey',
 'centurion',
 'vise',
 'maim',
 'virtual',
 'semicolon',
 'bream',
 'responsability',
 'lesser',
 'tcanning',
 'catalogue',
 'inc',
 'ipx',
 'streamflow',
 'schism',
 'rescorla',
 'admiral',
 'winmark',
 'offical',
 'johnn',
 'pendleton',
 'unscientific',
 'lewisville',
 'birdhouse',
 'bgm',
 'colin',
 'debater',
 'autobahn',
 'ariy',
 'wti',
 'multiplot',
 'scratchy',
 'hawaii',
 'hagerman',
 'oswald',
 'syrians',
 'county',
 'lgibb',
 'immer

In [221]:
len(vocab)

25736

In [222]:
vocab_dict = dict(zip(vocab, range(len(vocab))))

In [223]:
vocab_dict

{'mick': 0,
 'prominent': 1,
 'gaza': 2,
 'wellington': 3,
 'synchro': 4,
 'wb': 5,
 'samuel': 6,
 'dire': 7,
 'chmod': 8,
 'jmp': 9,
 'peg': 10,
 'armored': 11,
 'lamps': 12,
 'worshipper': 13,
 'atal': 14,
 'lunacy': 15,
 'reto': 16,
 'postet': 17,
 'teneis': 18,
 'sbradley': 19,
 'invasion': 20,
 'hobble': 21,
 'gallagher': 22,
 'creationists': 23,
 'gutting': 24,
 'hamishmar': 25,
 'workmanship': 26,
 'accurately': 27,
 'polex': 28,
 'seal': 29,
 'gqubv': 30,
 'classy': 31,
 'optpublisher': 32,
 'bxl': 33,
 'soso': 34,
 'experimental': 35,
 'inadvertently': 36,
 'harleys': 37,
 'lookit': 38,
 'cnes': 39,
 'decvax': 40,
 'asha': 41,
 'notch': 42,
 'teneo': 43,
 'anni': 44,
 'seeker': 45,
 'ballentine': 46,
 'hockey': 47,
 'centurion': 48,
 'vise': 49,
 'maim': 50,
 'virtual': 51,
 'semicolon': 52,
 'bream': 53,
 'responsability': 54,
 'lesser': 55,
 'tcanning': 56,
 'catalogue': 57,
 'inc': 58,
 'ipx': 59,
 'streamflow': 60,
 'schism': 61,
 'rescorla': 62,
 'admiral': 63,
 'winmark'

In [225]:
len(data['text'][1999])

35

In [226]:
data

,text,target,category
0,"[article, nazario, edgardo, nazario, write, in...",4,comp.sys.mac.hardware
1,"[deeply, grateful, citation, paper, electronic...",11,sci.crypt
2,"[system, key, appear, cryptographically, usele...",11,sci.crypt
3,"[mark, wayne, blunier, write, jimf, jim, frost...",7,rec.autos
4,"[article, tclock, tim, clock, write, article, ...",17,talk.politics.mideast
...,...,...,...
1995,"[interest, mail, list, bike, lifestyle, politi...",8,rec.motorcycles
1996,"[know, cause, black, border, edge, computer, s...",12,sci.electronics
1997,"[article, kdw, kenneth, whitehead, writes, day...",16,talk.politics.guns
1998,"[faq, list, program, window, thankx, info, bye...",5,comp.windows.x


In [227]:
data['text'] = data['text'].apply(lambda x: " ".join(x))

In [228]:
data

,text,target,category
0,article nazario edgardo nazario write info rum...,4,comp.sys.mac.hardware
1,deeply grateful citation paper electronic cash...,11,sci.crypt
2,system key appear cryptographically useless ch...,11,sci.crypt
3,mark wayne blunier write jimf jim frost write ...,7,rec.autos
4,article tclock tim clock write article amos am...,17,talk.politics.mideast
...,...,...,...
1995,interest mail list bike lifestyle politics mem...,8,rec.motorcycles
1996,know cause black border edge computer screen g...,12,sci.electronics
1997,article kdw kenneth whitehead writes day hear ...,16,talk.politics.guns
1998,faq list program window thankx info bye venky,5,comp.windows.x


In [229]:
vectorizer = TfidfVectorizer(vocabulary=vocab_dict)

In [230]:
tf_idf_matrix = vectorizer.fit_transform(data['text'])

In [231]:
tf_idf_matrix = tf_idf_matrix.toarray()

In [232]:
tf_idf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [233]:
df = pd.DataFrame(tf_idf_matrix)

In [235]:
df['target'] = data['target']
df['category'] = data['category']

In [236]:
df

,0,1,2,3,4,5,6,7,8,9,...,25728,25729,25730,25731,25732,25733,25734,25735,target,category
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,4,comp.sys.mac.hardware
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,11,sci.crypt
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.177025,0.0,0.0,0.0,11,sci.crypt
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.263409,0.0,0.0,0.000000,0.0,0.0,0.0,7,rec.autos
4,0.0,0.0,0.057476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,17,talk.politics.mideast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,8,rec.motorcycles
1996,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,12,sci.electronics
1997,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,16,talk.politics.guns
1998,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,5,comp.windows.x


In [237]:
pca = PCA(n_components=0.9)

In [238]:
reduced_tf_idf_matrix = pca.fit_transform(tf_idf_matrix)

In [239]:
reduced_tf_idf_matrix.shape

(2000, 1440)

In [240]:
reduced_tf_idf_matrix

array([[-1.49364251e-02, -4.23856877e-02,  1.55330318e-02, ...,
        -4.43029604e-02, -3.75572793e-02, -1.22249444e-03],
       [-1.48485369e-02, -3.98293143e-02, -1.29233428e-02, ...,
         5.11697053e-03, -2.83616524e-06,  1.28097814e-02],
       [-9.12681696e-04,  1.86347188e-01, -1.97910177e-01, ...,
        -1.80958771e-03, -1.52616279e-03, -3.54755084e-02],
       ...,
       [ 8.53453363e-02,  1.97543668e-02, -6.04398134e-02, ...,
        -2.94762683e-02, -1.87157775e-03, -1.21509306e-02],
       [-1.11809154e-01, -4.48283758e-03,  3.42934915e-02, ...,
         5.42621253e-03,  3.17086517e-03,  1.92180945e-02],
       [ 4.08384039e-03, -5.13953796e-02, -3.35192444e-03, ...,
        -1.12597875e-02,  3.42421109e-03,  2.27040183e-03]])

In [241]:
df = pd.DataFrame(data=reduced_tf_idf_matrix)

In [242]:
df['target'] = data['target']
df['category'] = data['category']

In [243]:
df

,0,1,2,3,4,5,6,7,8,9,...,1432,1433,1434,1435,1436,1437,1438,1439,target,category
0,-0.014936,-0.042386,0.015533,0.013352,-0.031512,-0.021308,0.025912,0.027292,2.858378e-02,0.006644,...,0.014956,0.021195,-0.018150,0.013523,0.012625,-0.044303,-0.037557,-0.001222,4,comp.sys.mac.hardware
1,-0.014849,-0.039829,-0.012923,-0.010453,-0.043544,-0.041542,-0.005264,0.031471,-5.219824e-03,-0.022679,...,0.010801,-0.011351,-0.003104,0.001941,-0.003737,0.005117,-0.000003,0.012810,11,sci.crypt
2,-0.000913,0.186347,-0.197910,-0.049676,0.084674,-0.166164,0.041899,0.062024,4.771921e-02,-0.009355,...,-0.001736,0.009643,0.004162,-0.010720,-0.001146,-0.001810,-0.001526,-0.035476,11,sci.crypt
3,-0.004022,-0.006643,0.022866,0.199507,-0.010042,-0.001115,-0.045336,-0.060955,-9.520519e-03,0.018248,...,0.006402,-0.013173,0.007156,0.021445,-0.018900,0.000152,0.037220,0.012348,7,rec.autos
4,0.097669,0.023057,-0.008501,0.023613,-0.044812,0.057996,-0.012246,-0.041575,3.215903e-02,-0.110776,...,0.010314,0.011221,0.014652,-0.019330,0.010554,-0.001168,-0.029919,-0.018488,17,talk.politics.mideast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.066929,-0.015765,-0.004160,-0.045855,-0.063707,-0.036876,-0.046725,0.004559,-8.493001e-02,0.023850,...,-0.004578,-0.012421,-0.031378,0.027032,0.004628,0.005381,-0.029053,-0.000469,8,rec.motorcycles
1996,-0.050045,-0.004259,0.007085,0.008336,0.022375,0.053042,0.086772,-0.082858,-1.098443e-02,0.077079,...,0.005221,-0.000367,0.015348,0.018527,-0.006860,-0.044071,0.026079,-0.006775,12,sci.electronics
1997,0.085345,0.019754,-0.060440,0.037299,-0.045391,0.096453,0.005844,0.001931,-3.096398e-02,0.032864,...,-0.045833,-0.006944,0.030748,0.006405,0.012959,-0.029476,-0.001872,-0.012151,16,talk.politics.guns
1998,-0.111809,-0.004483,0.034293,-0.120170,-0.027195,0.031656,-0.049169,0.023485,-4.453917e-07,-0.071679,...,0.006112,0.004080,-0.007796,0.002982,0.022493,0.005426,0.003171,0.019218,5,comp.windows.x
